# Exploring DeFi Concepts: Token Wrapping, Staking & Arbitrage Signals


This notebook explores simplified decentralized finance (DeFi) mechanisms using Python and Solidity.
Covered topics:

- Basic transactions on Ethereum testnets

- Smart contract deployment and interaction

- Simulated ERC-20 tokens and wrapped tokens

- Chainlink oracle integration



In [1]:
from web3 import Web3
import solcx
import os
from dotenv import load_dotenv
import json
from eth_typing import ChecksumAddress

### 1. Ethereum transaction simulation

This cell demonstrates a basic local Ethereum transaction using Web3.py and the EthereumTesterProvider. It checks account balances, sends ETH from one account to another, and prints the transaction details.

In [ ]:
# Connect to local Ethereum tester network
w3 = Web3(Web3.EthereumTesterProvider())

assert w3.is_connected(), "Connection failed"

# View initial balances
account_0 = w3.eth.accounts[0]
account_1 = w3.eth.accounts[1]

print(f"Account[0] initial balance: {Web3.from_wei(w3.eth.get_balance(account_0), 'ether')} ETH")
print(f"Account[1] initial balance: {Web3.from_wei(w3.eth.get_balance(account_1), 'ether')} ETH")

# Send ETH from account_0 to account_1
tx = {
    'from': account_0,
    'to': account_1,
    'value': Web3.to_wei(3, 'ether'),
    'gas': 21000,
    'gasPrice': Web3.to_wei(20, 'gwei')
}
tx_hash = w3.eth.send_transaction(tx)

# Get receipt and display results
receipt = w3.eth.get_transaction_receipt(tx_hash)
new_balance = w3.eth.get_balance(account_0)

print(f"Transaction hash: {tx_hash.hex()}")
print(f"New Account[0] balance: {Web3.from_wei(new_balance, 'ether')} ETH")
print(f"Account[1] balance: {Web3.from_wei(w3.eth.get_balance(account_1), 'ether')} ETH")
print(f"Transaction receipt:\n{receipt}")

Account[0] initial balance: 1000000 ETH
Account[1] initial balance: 1000000 ETH
Transaction hash: b2e6e6c3bbaf3a46e607ffeb3398fb0e1a1a9c168dc2e3edebb125b0b8a67f7d
New Account[0] balance: 999996.99958 ETH
Account[1] balance: 1000003 ETH
Transaction receipt:
AttributeDict({'blockHash': HexBytes('0x454721a1c832f2ecc291fbad96749bc1da0c9e2ede92d3fbb9125b13eab496a6'), 'blockNumber': 1, 'contractAddress': None, 'cumulativeGasUsed': 21000, 'effectiveGasPrice': 20000000000, 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf', 'gasUsed': 21000, 'logs': [], 'state_root': b'\x01', 'status': 1, 'to': '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF', 'transactionHash': HexBytes('0xb2e6e6c3bbaf3a46e607ffeb3398fb0e1a1a9c168dc2e3edebb125b0b8a67f7d'), 'transactionIndex': 0, 'type': 0})


## Concept 1: Lock-and-Mint Simulation (Wrapped Tokens)

In [ ]:
solcx.install_solc('0.8.0') 
solcx.set_solc_version('0.8.0')

In [ ]:
# Solidity code for a simple storage contract
# This contract allows storing a favorite number and associating names with numbers.
# It also provides functions to retrieve the stored number and add people with their favorite numbers.  \
    
solidity_code = ''' 
pragma solidity >=0.6.0 <0.9.0;

contract SimpleStorage {

    uint256 favoriteNumber;

    struct People {
        uint256 favoriteNumber;
        string name;
    }

    People[] public people;
    mapping(string => uint256) public nameToFavoriteNumber;

    function store(uint256 _favoriteNumber) public {
        favoriteNumber = _favoriteNumber;
    }
    
    function retrieve() public view returns (uint256){
        return favoriteNumber;
    }

    function addPerson(string memory _name, uint256 _favoriteNumber) public {
        people.push(People(_favoriteNumber, _name));
        nameToFavoriteNumber[_name] = _favoriteNumber;
    }
}
'''

# Compile the Solidity code

compiled_sol = solcx.compile_source(
    solidity_code,
    output_values=["abi", "bin"]
)

# Get contract interface
contract_id, contract_interface = compiled_sol.popitem()

# Access ABI and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

# Print the ABI for verification
print("Contract ABI:")  
from pprint import pprint
pprint(abi)


In [ ]:
# Set default account for transactions
w3.eth.default_account = w3.eth.accounts[0]

# Create contract in Python
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

# Deploy the contract
tx_hash = SimpleStorage.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Get the deployed contract instance
contract_instance = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

print("Contract deployed at:", tx_receipt.contractAddress)

# Call store() to set favoriteNumber = 42
tx_hash = contract_instance.functions.store(42).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Call retrieve() to get it back
retrieved_number = contract_instance.functions.retrieve().call()
print("Retrieved favorite number:", retrieved_number)

# Add a person to the array
tx_hash = contract_instance.functions.addPerson("Alice", 99).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Check the mapping
fav_number = contract_instance.functions.nameToFavoriteNumber("Alice").call()
print("Alice's favorite number:", fav_number)

# Access the first entry in the people array
person = contract_instance.functions.people(0).call()
print("Person stored:", person)

In [ ]:
# a simple oracle usage example
# This contract fetches the latest ETH/USD price from a Chainlink oracle.
# Needs to be deployed on a testnet Sepolia

solidity_code_oracle = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@chainlink/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol";

contract PriceConsumerV3 {
    AggregatorV3Interface internal priceFeed;

    constructor() {
        // ETH/USD on Sepolia
        priceFeed = AggregatorV3Interface(
            0x694AA1769357215DE4FAC081bf1f309aDC325306
        );
    }

    function getLatestPrice() public view returns (int) {
        (, int price,,,) = priceFeed.latestRoundData();
        return price;
    }
}
'''

# Connect to Sepolia

load_dotenv()

infura_key = os.getenv("INFURA_KEY")
private_key = os.getenv("PRIVATE_KEY")

w3 = Web3(Web3.HTTPProvider("https://sepolia.infura.io/v3/infura_key"))
print(w3.is_connected())
account = w3.eth.account.from_key(private_key)
w3.eth.default_account = account.address

# Compile
solcx.install_solc("0.8.0")
compiled = solcx.compile_source(solidity_code_oracle, output_values=["abi", "bin"])
contract_id, interface = compiled.popitem()
abi = interface["abi"]
bytecode = interface["bin"]

# Deploy
contract = w3.eth.contract(abi=abi, bytecode=bytecode)
tx = contract.constructor().build_transaction({
    "nonce": w3.eth.get_transaction_count(account.address),
    "gas": 300000,
    "gasPrice": w3.to_wei("20", "gwei")
})
signed_tx = w3.eth.account.sign_transaction(tx, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print("Deployed at:", tx_receipt.contractAddress)

# Interact
deployed = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
price = deployed.functions.getLatestPrice().call()
print("Latest ETH/USD price:", price / 1e8)


In [10]:
# First, let's create a simple mock ERC20 token
mock_token_code = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract MockToken {
    mapping(address => uint256) private _balances;
    mapping(address => mapping(address => uint256)) private _allowances;
    uint256 private _totalSupply;
    
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
    
    constructor() {
        _mint(msg.sender, 1000000 * 10**18); // Mint 1M tokens to deployer
    }
    
    function transfer(address to, uint256 amount) public returns (bool) {
        _transfer(msg.sender, to, amount);
        return true;
    }
    
    function approve(address spender, uint256 amount) public returns (bool) {
        _approve(msg.sender, spender, amount);
        return true;
    }
    
    function transferFrom(address from, address to, uint256 amount) public returns (bool) {
        _spendAllowance(from, msg.sender, amount);
        _transfer(from, to, amount);
        return true;
    }
    
    function _transfer(address from, address to, uint256 amount) internal {
        require(from != address(0), "Transfer from zero");
        require(to != address(0), "Transfer to zero");
        require(_balances[from] >= amount, "Insufficient balance");
        
        _balances[from] -= amount;
        _balances[to] += amount;
        emit Transfer(from, to, amount);
    }
    
    function _approve(address owner, address spender, uint256 amount) internal {
        _allowances[owner][spender] = amount;
        emit Approval(owner, spender, amount);
    }
    
    function _spendAllowance(address owner, address spender, uint256 amount) internal {
        require(_allowances[owner][spender] >= amount, "Insufficient allowance");
        _allowances[owner][spender] -= amount;
    }
    
    function _mint(address account, uint256 amount) internal {
        _totalSupply += amount;
        _balances[account] += amount;
        emit Transfer(address(0), account, amount);
    }
    
    // View functions
    function balanceOf(address account) public view returns (uint256) {
        return _balances[account];
    }
    
    function allowance(address owner, address spender) public view returns (uint256) {
        return _allowances[owner][spender];
    }
    
    function totalSupply() public view returns (uint256) {
        return _totalSupply;
    }
}
'''

# Now a simplified wrapped token without external dependencies
simplified_wrapped_token = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract WrappedToken {
    address public immutable underlying;
    mapping(address => uint256) private _balances;
    uint256 private _totalSupply;
    bool public paused;
    address public owner;
    
    event Deposit(address indexed user, uint256 amount);
    event Withdraw(address indexed user, uint256 amount);
    
    modifier onlyOwner() {
        require(msg.sender == owner, "Not owner");
        _;
    }
    
    constructor(address _underlying) {
        underlying = _underlying;
        owner = msg.sender;
    }
    
    function deposit(uint256 amount) external {
        require(!paused, "Paused");
        require(amount > 0, "Zero amount");
        
        // Transfer tokens using low-level call
        (bool success, bytes memory data) = underlying.call(
            abi.encodeWithSignature(
                "transferFrom(address,address,uint256)", 
                msg.sender, 
                address(this), 
                amount
            )
        );
        require(success && (data.length == 0 || abi.decode(data, (bool))), "Transfer failed");
        
        _balances[msg.sender] += amount;
        _totalSupply += amount;
        emit Deposit(msg.sender, amount);
    }
    
    function withdraw(uint256 amount) external {
        require(!paused, "Paused");
        require(_balances[msg.sender] >= amount, "Insufficient");
        
        _balances[msg.sender] -= amount;
        _totalSupply -= amount;
        
        (bool success, bytes memory data) = underlying.call(
            abi.encodeWithSignature(
                "transfer(address,uint256)",
                msg.sender,
                amount
            )
        );
        require(success && (data.length == 0 || abi.decode(data, (bool))), "Transfer failed");
        
        emit Withdraw(msg.sender, amount);
    }
    
    function balanceOf(address account) external view returns (uint256) {
        return _balances[account];
    }
    
    function totalSupply() external view returns (uint256) {
        return _totalSupply;
    }
    
    function pause() external onlyOwner {
        paused = true;
    }
    
    function unpause() external onlyOwner {
        paused = false;
    }
}
'''

In [12]:
# Reset the Web3 instance to use local test provider
w3 = Web3(Web3.EthereumTesterProvider())
w3.eth.default_account = w3.eth.accounts[0]

# Compile and deploy both contracts
mock_token = w3.eth.contract(
    abi=solcx.compile_source(mock_token_code)['<stdin>:MockToken']['abi'],
    bytecode=solcx.compile_source(mock_token_code)['<stdin>:MockToken']['bin']
)

# Deploy MockToken
tx_hash = mock_token.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
mock_token_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=mock_token.abi)

# Deploy WrappedToken with MockToken address
wrapped_token = w3.eth.contract(
    abi=solcx.compile_source(simplified_wrapped_token)['<stdin>:WrappedToken']['abi'],
    bytecode=solcx.compile_source(simplified_wrapped_token)['<stdin>:WrappedToken']['bin']
)

tx_hash = wrapped_token.constructor(mock_token_instance.address).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
wrapped_token_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=wrapped_token.abi)

# Test wrapping flow
print("Initial balance:", mock_token_instance.functions.balanceOf(w3.eth.accounts[0]).call())

# Approve wrapped token contract to spend our tokens
tx_hash = mock_token_instance.functions.approve(
    wrapped_token_instance.address, 
    1000 * 10**18
).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Deposit tokens
amount = 100 * 10**18  # 100 tokens
tx_hash = wrapped_token_instance.functions.deposit(amount).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

print("Wrapped token balance:", wrapped_token_instance.functions.balanceOf(w3.eth.accounts[0]).call())

Initial balance: 1000000000000000000000000
Wrapped token balance: 100000000000000000000
